In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os, json
import numpy as np
from argparse import ArgumentParser
from tqdm import tqdm
from collections import defaultdict
import pandas as pd
from pprint import pprint
from datetime import datetime
import copy

# Output Extraction for Zero-shot models
check the outputs of LLaMA models, preprocess the output and saving it.

In [ ]:
def read_json_file(name):
    with open(name, 'r') as f:
        data = json.load(f)
        return data

def save_json(json_data, file_name):
    json_data = json.dumps(json_data)
    print(file_name)
    with open(file_name, "w") as json_file:
          json_file.write(json_data)

##GPT-4
Writing the whole thing GPT-4 model again. This will be mostly similar to other parts of the code.

In [ ]:
## These are the processing files.

#code to find specific values of the arguments in a string
def GPT_find_batch_arg_value(arg, string_to_check):
    start_index = string_to_check.find(arg)

    if start_index != -1:
        finding_char = string_to_check[start_index-1] #finding that character between which arguments are encapsulated
        start_index += len(arg) + 2

        first_index = string_to_check.find(finding_char, start_index)+1
        newline_comma_index = max(string_to_check.find(',', start_index), string_to_check.find('\n', start_index))

        if(newline_comma_index<=first_index and newline_comma_index!=-1): #newline or comma happens before first index this value is null.
            value_list = ['null']
            return value_list
        end_index = string_to_check.find(finding_char, first_index)

        value = string_to_check[first_index:end_index]
        # Split the value into a list based on commas
        value_list = [item.replace("'", "").strip() for item in value.split(',')]
    else:
        value_list = ['null'] #if an argument is not present returning null.
    return value_list

def GPT_batch_arguments_extraction(raw_preds, arguments):
    extracted_preds = {}
    start_index = raw_preds.find('{') + 1
    end_index = raw_preds.find('}', start_index)
    # print(raw_preds)
    string_of_interest = raw_preds[start_index:end_index] #most outpt is between {}. So this is our string of interest.
    # print(string_of_interest)
    for arg in arguments:
        value_list = GPT_find_batch_arg_value(arg, string_of_interest)
        extracted_preds[arg]= copy.deepcopy(value_list)
        #print(value_list)
    return extracted_preds ##return the argument values for each of the role.


In [ ]:
folder_path = 'Argument-Extraction-Predictions'
file_name = '1-gpt-4o_test_100_predictions.json'
results = read_json_file(os.path.join(folder_path, file_name))


In [ ]:
results.keys()
results['batch_description_guided'][0].keys()

dict_keys(['doc_id', 'id', 'text1', 'text2', 'label', 'thread-length', 'ground-truth-arguments', 'raw-predictions'])

In [ ]:
def create_prediction_file(nm):
    print(file_name)
    folder_path = 'Argument-Extraction-Predictions'
    results = read_json_file(os.path.join(folder_path, file_name))
    event_roles = read_json_file(os.path.join('role_definitions.json'))


    prompt_types = ['batch'] #['batch', 'isolated'] %for GPT-4 we have only experimented with batch type.
    exp_types = ['description_guided', 'question_guided']
    arg_types = ['main-arguments', 'event-specific-arguments', 'subject-effect-arguments']

    data_lst = results[f'batch_description_guided'] #Just taking the values from one type. All others are same.

    print(list(data_lst[0].keys()))

    #the whole prediction preparaion guideline is here
    predictions_1 = []
    for dt in data_lst:
        new_dict = {key: value for key, value in dt.items() if key != 'raw-predictions'}
        predictions_1.append(new_dict) #predictions_1 has all the values other than raw predictions.

    print(list(predictions_1[0].keys()))
    #raw-predictions is a dictionary of predictions
    predictions_2 = []
    for i in range(len(predictions_1)):
        data_dict = predictions_1[i]
        for p_type in prompt_types:
            for e_type in exp_types:
                raw_pred = results[f'{p_type}_{e_type}'][i]['raw-predictions']
                data_dict[f'raw-{p_type}_{e_type}-predictions'] = copy.deepcopy(raw_pred)
        #pprint(data_dict)
        predictions_2.append(data_dict)
    print(list(predictions_2[0].keys()))

    processed_predictions = []
    for i in range(len(predictions_2)):
        data_dict = predictions_2[i]
        for p_type in prompt_types:
              for e_type in exp_types:
                  event = data_dict['label']
                  ground_truth = data_dict['ground-truth-arguments']
                  raw_pred = data_dict[f'raw-{p_type}_{e_type}-predictions'] ## raw predictions from LLM
                  print(data_dict['doc_id'], p_type, e_type)
                  print("ground truth")
                  pprint(ground_truth)
                  if p_type =='batch': ##
                      preds = {}
                      for arg_typ in arg_types: ## going over each main, event-specific and subject-effect arguments.
                          arguments = list(event_roles[event][arg_typ].keys())
                          #print(p_type, e_type, arg_typ)
                          val = GPT_batch_arguments_extraction(raw_pred[arg_typ], arguments) #calling corresponding functions to clean data
                          preds[arg_typ] = copy.deepcopy(val) ##saving the values.
                      data_dict[f'processed-{p_type}_{e_type}-predictions']=copy.deepcopy(preds)

                  print("predictions")
                  pprint(data_dict[f'processed-{p_type}_{e_type}-predictions'])
        # pprint(data_dict)
        processed_predictions.append(data_dict)
    print(list(processed_predictions[0].keys()))
    return processed_predictions

In [ ]:
numbers = ['1', '2']
for number in numbers:
    name = 'gpt-4o_test_100_predictions'
    file_name = f'{number}-{name}.json'
    create_prediction_file(file_name)
    processed_predictions = create_prediction_file(file_name)
    print(len(processed_predictions))
    folder_path = 'rgument-Extraction-Predictions'
    file_name = f'{number}-processed-{name}.json'
    save_json(processed_predictions, os.path.join(folder_path, file_name))


Streaming output truncated to the last 5000 lines.
 'subject-effect-arguments': {'age': ['null'],
                              'conditions': ['spinal fusion',
                                             'denied pain management'],
                              'gender': ['null'],
                              'intervention': ['suboxone treatment', 'AA'],
                              'severity': ['high'],
                              'side-effect-duration': ['11 years'],
                              'side-effects': ['failed 7 day subutex detox'],
                              'start-time': ['null']}}
predictions
{'event-specific-arguments': {'dosage': ['2mg', '4mg', '6mg'],
                              'frequency': ['daily'],
                              'manner': ['between gums and lip', 'plugged'],
                              'medications': ['subs'],
                              'purpose': [''],
                              'timing': ['null'],
                              '

# Batch Extraction for all the models
Batch extraction will be the same for all other models

In [ ]:
## These are the processing files.

#code to find specific values of the arguments in a string
def find_batch_arg_value(arg, string_to_check):
    start_index = string_to_check.find(arg)

    if start_index != -1:
        finding_char = string_to_check[start_index-1] #finding that character between which arguments are encapsulated
        start_index += len(arg) + 2

        first_index = string_to_check.find(finding_char, start_index)+1
        newline_comma_index = max(string_to_check.find(',', start_index), string_to_check.find('\n', start_index))

        if(newline_comma_index<=first_index and newline_comma_index!=-1): #newline or comma happens before first index this value is null.
            value_list = ['null']
            return value_list
        end_index = string_to_check.find(finding_char, first_index)

        value = string_to_check[first_index:end_index]
        # Split the value into a list based on commas
        value_list = [item.replace("'", "").strip() for item in value.split(',')]
    else:
        value_list = ['null'] #if an argument is not present returning null.
    return value_list

def batch_arguments_extraction(raw_preds, arguments):
    extracted_preds = {}
    start_index = raw_preds.find('{') + 1
    end_index = raw_preds.find('}', start_index)
    string_of_interest = raw_preds[start_index:end_index] #most outpt is between {}. So this is our string of interest.
    for arg in arguments:
        value_list = find_batch_arg_value(arg, string_of_interest)
        extracted_preds[arg]= copy.deepcopy(value_list)
    return extracted_preds

# def isolated_arguments_extraction(raw_preds, argument):
#     start_index = raw_preds.find(']') + 1
#     end_index = raw_preds.find('[', start_index)
#     value = raw_preds[start_index:end_index]

#     find_assistant = value.find('assistant') ## if the extracted string the has assistatnt then I parse wrongly and it it should be null.
#     if find_assistant==-1:
#         value_list = [item.replace("'", "").strip() for item in value.split(',')]
#     else: value_list = ['null']
#     return value_list


In [ ]:
def create_prediction_file(nm):
    print(file_name)
    folder_path = '/content/drive/Shareddrives/Reddit Misinformation/Data/New-Avg-Run-Experiments/Argument-Extraction-Predictions'
    results = read_json_file(os.path.join(folder_path, file_name))
    event_roles = read_json_file(os.path.join('/content/drive/Shareddrives/Reddit Misinformation/Data/Final-Data/role_definitions.json'))


    prompt_types = ['batch']
    exp_types = ['description_guided', 'question_guided']
    arg_types = ['main-arguments', 'event-specific-arguments', 'subject-effect-arguments']

    data_lst = results[f'batch_description_guided'] #Just taking the values from one type. All others are same.

    #the whole prediction preparaion guideline is here
    predictions_1 = []
    for dt in data_lst:
        new_dict = {key: value for key, value in dt.items() if key != 'raw-predictions'}
        predictions_1.append(new_dict)

    predictions_2 = []
    for i in range(len(predictions_1)):
        data_dict = predictions_1[i]
        #print(data_dict)
        for p_type in prompt_types:
            for e_type in exp_types:
                raw_pred = results[f'{p_type}_{e_type}'][i]['raw-predictions']
                data_dict[f'raw-{p_type}_{e_type}-predictions'] = copy.deepcopy(raw_pred)
        #pprint(data_dict)
        predictions_2.append(data_dict)

    processed_predictions = []
    for i in range(len(predictions_2)):
        data_dict = predictions_2[i]
        for p_type in prompt_types:
              for e_type in exp_types:
                  event = data_dict['label']
                  ground_truth = data_dict['ground-truth-arguments']
                  raw_pred = data_dict[f'raw-{p_type}_{e_type}-predictions'] ## raw predictions from LLM

                  if p_type =='batch': ##
                      preds = {}
                      for arg_typ in arg_types: ## going over each main, event-specific and subject-effect arguments.
                          arguments = list(event_roles[event][arg_typ].keys())
                          #print(p_type, e_type, arg_typ)
                          val = batch_arguments_extraction(raw_pred[arg_typ], arguments) #calling corresponding functions to clean data
                          preds[arg_typ] = copy.deepcopy(val) ##saving the values.
                      data_dict[f'processed-{p_type}_{e_type}-predictions']=copy.deepcopy(preds)

                  # if p_type == 'isolated':
                  #     preds = {}
                  #     for arg_typ in arg_types: ## going over each main, event-specific and subject-effect arguments.
                  #           preds[arg_typ] = {}
                  #           arguments = list(event_roles[event][arg_typ].keys())
                  #           for arrg in arguments:
                  #               #print(p_type, e_type, arg_typ, arrg)
                  #               #print(raw_pred[arg_typ][arrg])
                  #               val = isolated_arguments_extraction(raw_pred[arg_typ][arrg], arrg) #calling corresponding functions to clean data
                  #               # print(val)
                  #               preds[arg_typ][arrg] = copy.deepcopy(val) ##saving the values.
                  #     data_dict[f'processed-{p_type}_{e_type}-predictions']=copy.deepcopy(preds)

                  print(data_dict['doc_id'], p_type, e_type)
                  pprint(data_dict[f'processed-{p_type}_{e_type}-predictions'])
        # pprint(data_dict)
        processed_predictions.append(data_dict)

    return processed_predictions



In [ ]:
number = ['1', '2']
model_names = ['Llama-3-70B-Instruct', 'Llama-3-8B-Instruct', 'Mixtral-8x7B-Instruct', 'gemma-1.1-7b-it']

for mdl_name in model_names:
    print(mdl_name)
    for number in numbers:
        name = f'{mdl_name}_test_100_predictions'
        file_name = f'{number}-{name}.json'
        create_prediction_file(file_name)
        processed_predictions = create_prediction_file(file_name)
        print(len(processed_predictions))
        folder_path = '/content/drive/Shareddrives/Reddit Misinformation/Data/New-Avg-Run-Experiments/Argument-Extraction-Predictions'
        file_name = f'{number}-processed-{name}.json'
        print(file_name)
        save_json(processed_predictions, os.path.join(folder_path, file_name))


Streaming output truncated to the last 5000 lines.
                              'side-effect-duration': ['null'],
                              'side-effects': ['Unknown'],
                              'start-time': ['null']}}
doc_32_Madhu batch description_guided
{'event-specific-arguments': {'condition': ['Tapering-induced brain fog and '
                                            'anxiety'],
                              'current-dosage': ['2mg'],
                              'goal-dosage': ['Zero'],
                              'initial-dosage': ['Unknown'],
                              'start-time': ['Unknown'],
                              'taper-medications': ['Suboxone'],
                              'target-duration': ['Unknown'],
                              'trigger': ['Factors contributing to tapering'],
                              'type': ['Self-tapering']},
 'main-arguments': {'effects': ['Anxiety', 'brain fog', 'existential feelings'],
                    'sub